In [1]:
import urllib.request
import re
import numpy as np
import pandas as pd
import json


In [26]:
df = pd.read_csv('data/all_users_2015.csv')
df['index'] = range(len(df))
df = pd.concat([df.iloc[:,-1],df.iloc[:,:-1] ], axis=1) # Move index to 1st column
df.head()

,index,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,created_at,...,profile_background_image_url,profile_background_color,profile_link_color,utc_offset,protected,verified,description,updated,dataset,account_type
0,0,24503,Roberto Bonanzinga,Bonanzinga,4340,5055,1466,267,256,Mon Nov 27 06:55:12 +0000 2006,...,http://a0.twimg.com/profile_background_images/...,352726,D02B55,-28800.0,NaN,NaN,Partner at Balderton Capital (formerly Benchma...,14/2/2015 11:32,TFP,real
1,1,22903,effeffe,effeffe,164,132,194,12,4,Sun Nov 26 15:19:32 +0000 2006,...,http://a0.twimg.com/images/themes/theme14/bg.gif,131516,9999,3600.0,NaN,NaN,L'uomo ha creato dio a sua immagine e somiglia...,14/2/2015 11:32,TFP,real
2,2,382393,Ciro Cattuto,ciro,1070,1154,832,871,92,Sun Dec 31 02:03:17 +0000 2006,...,http://a0.twimg.com/images/themes/theme2/bg.gif,C6E2EE,1F98C7,3600.0,NaN,NaN,Research Director at ISI Foundation. Data scie...,14/2/2015 11:32,TFP,real
3,3,286543,Alessio Bragadini,abragad,6892,930,535,478,28,Wed Dec 27 14:55:17 +0000 2006,...,http://a0.twimg.com/profile_background_images/...,9AE4E8,0000FF,3600.0,NaN,NaN,Web and social media developer from Italy,14/2/2015 11:32,TFP,real
4,4,438023,fullcaffeine,fullcaffeine,2885,173,444,41,2,Tue Jan 02 09:01:50 +0000 2007,...,http://a0.twimg.com/images/themes/theme14/bg.gif,131516,9999,3600.0,NaN,NaN,NaN,14/2/2015 11:32,TFP,real


In [25]:
df_2017 = pd.read_excel('data/data_2017.xlsx')
df_2017.shape

(14368, 43)

In [4]:
check_columns = pd.concat([pd.Series(df_2017.columns),pd.Series(df.columns[1:])],axis=1)

check_columns

,0,1
0,id,id
1,name,name
2,screen_name,screen_name
3,statuses_count,statuses_count
4,followers_count,followers_count
5,friends_count,friends_count
6,favourites_count,favourites_count
7,listed_count,listed_count
8,url,created_at
9,lang,url


In [29]:
def edit_url(url):
    starting = url[:16]
    pattern = 'a\d'
    if re.search(pattern, starting):
        replaced_pbs = re.sub(pattern, "pbs", starting)
    else:
        replaced_pbs = starting
    edited = replaced_pbs + url[16:]
    return edited

In [39]:
# preprocess url to replace with working url
for index, row in df_2017.iterrows():
    image_url = row['profile_image_url']
    df_2017.loc[index, 'edited_profile_image_url'] = edit_url(str(image_url))

In [41]:
failed_id = []
successful_id = []

for index, row in df_2017.iterrows():
    id = row['id']
    imgURL = row['edited_profile_image_url']
    try:
        conn = urllib.request.urlopen(imgURL)
    except Exception as e:
        failed_id.append(id)
    else :
        try:
            urllib.request.urlretrieve(imgURL, "new_batch_2017/{}.jpg".format(id))
        except Exception as e:
            failed_id.append(id)
        else:
            successful_id.append(id)
    print(index/len(df_2017) * 100)

0.0
0.006959910913140311
0.013919821826280623
0.020879732739420936
0.027839643652561245
0.03479955456570156
0.04175946547884187
0.048719376391982185
0.05567928730512249
0.0626391982182628
0.06959910913140312
0.07655902004454343
0.08351893095768374
0.09047884187082406
0.09743875278396437
0.10439866369710468
0.11135857461024498
0.11831848552338531
0.1252783964365256
0.1322383073496659
0.13919821826280623
0.14615812917594656
0.15311804008908686
0.16007795100222716
0.16703786191536749
0.1739977728285078
0.1809576837416481
0.1879175946547884
0.19487750556792874
0.20183741648106907
0.20879732739420936
0.21575723830734966
0.22271714922048996
0.2296770601336303
0.23663697104677062
0.24359688195991092
0.2505567928730512
0.2575167037861915
0.2644766146993318
0.27143652561247217
0.27839643652561247
0.28535634743875277
0.2923162583518931
0.2992761692650334
0.3062360801781737
0.313195991091314
0.3201559020044543
0.3271158129175947
0.33407572383073497
0.34103563474387527
0.3479955456570156
0.3549554

In [42]:
print(len(failed_id))
print(len(successful_id))

3961
10407


In [43]:
# with open("failed_id_new_batch_2017.json", "w") as output:
#     json.dump(failed_id, output, indent=2) 
    
# with open("successful_id_new_batch_2017.json", "w") as output:
#     json.dump(successful_id, output, indent=2) 